Задачи:
* Добавление классов в датасет
* Оценка показателей классификации
* Анализ ошибок классификатора
* Дополнение обучения классификации аугментациями https://albumentations.ai/
* Дополнение вывода метрик при обучении в Tensorboard train: loss, valid: calssification metrics (по каждому классу и усредненные по macro), loss, а также lr


Инклудим все подряд, потому что можем

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random
import torch
from torch import nn
import seaborn as sns
import pandas as pd
import os
import pathlib
import shutil
import cv2
import PIL
import cv2
from datetime import datetime

%cd adas_system/notebooks

IN_COLAB = False
USE_COLAB_GPU = False
try:
    import google.colab
    IN_COLAB = True
    USE_COLAB_GPU = True
    from google.colab import drive
    drive.mount('/content/drive')
    if not os.path.isfile('1_ClassifierResearch.ipynb'):
        !git clone --branch 9_SignDetector https://github.com/lsd-maddrive/adas_system.git

    !gdown --id 1-K3ee1NbMmx_0T5uwMesStmKnZO_6mWi
    %cd adas_system/notebooks
    !mkdir ../data/R_MERGED
    !unzip -q -o /content/R_MERGED.zip -d ./../data/

except:
    if IN_COLAB:
        print('[!]YOU ARE IN COLAB, BUT DIDNT MOUND A DRIVE. Model wont be synced[!]')

        if not os.path.isfile('1_ClassifierResearch.ipynb'):
            !git clone --branch 9_SignDetector https://github.com/lsd-maddrive/adas_system.git
            !gdown --id 1-K3ee1NbMmx_0T5uwMesStmKnZO_6mWi
            %cd adas_system/notebooks
            !mkdir ../data/R_MERGED
            !unzip -q -o /content/R_MERGED.zip -d ./../data/

        IN_COLAB = False

    else:
        pass

###
import nt_helper
from nt_helper.helper_utils import *
###

TEXT_COLOR = 'black'

# Зафиксируем состояние случайных чисел
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)
random.seed(RANDOM_STATE)
%matplotlib inline
plt.rcParams["figure.figsize"] = (17,10)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
if not IN_COLAB:
    PROJECT_ROOT = pathlib.Path(os.path.join(os.curdir, os.pardir))
else:
    PROJECT_ROOT = pathlib.Path('..')
    
DATA_DIR = PROJECT_ROOT / 'data'
NOTEBOOKS_DIR = PROJECT_ROOT / 'notebooks'

1. Добавить инфы в датасет. 

> RTSD Public не содержит 14 необходимых знаков: 1.6, 1.31, 2.4, 3.21, 3.22, 3.23. 3.24 (90, 100, 110, 120, 130), 3.25, 3.31, 6.3.2.

| Знак | Описание | Источник |
| ------------- | ------------- | ---- |
| 1.6 | Пересечение равнозначных дорог | - |
| 1.31 | Туннель | - |
| 2.4 | Уступите дорогу | GTSRB Recognition |
| 3.21 | Конец запрещения обгона | GTSRB Recognition |
| 3.22 | Обгон грузовым автомобилям запрещен | GTSRB Recognition |
| 3.23 | Конец запрещения обгона грузовым автомобилям | GTSRB Recognition |
| 3.24-90 | Огр 90 | - |
| 3.24-100 | Огр 100 | GTSRB Recognition |
| 3.24-110 | Огр 110 | - |
| 3.24-120 | Огр 120 | GTSRB Recognition |
| 3.24-130 | Огр 130 | - |
| 3.25 | Конец огр. максимальной скорости | GTSRB Recognition |
| 3.31 | Конец всех ограничений | GTSRB Recognition |
| 6.3.2 | Зона для разворота | - |


Идея: берем стандартные изображения знаков, обучаем классификатор на них

In [ ]:
import pandas as pd
import os
import pathlib
import shutil
import numpy as np

STOCK_SIGNS_CSV_LOCATION = DATA_DIR / 'STOCK_SIGNS.csv'
PREFIX = 'STOCK_SIGNS\\'
STOCK_SIGNS_DIR = pathlib.Path('D:\\d_tsw\\1_SIGN_CLASSIFIER\\data\\STOCK_SIGNS')
STOCK_SIGNS_LIST = os.listdir(STOCK_SIGNS_DIR)

STOCK_SIGNS_DATAFRAME = pd.DataFrame(columns=['filepath', 'SIGN'])
for sign in STOCK_SIGNS_LIST:
    sign_as_series = pd.DataFrame(data=[[PREFIX + sign, '.'.join(sign.split('.')[:-1])]], 
                                  columns=STOCK_SIGNS_DATAFRAME.columns
                                 )
    STOCK_SIGNS_DATAFRAME = STOCK_SIGNS_DATAFRAME.append(sign_as_series, ignore_index=True)

STOCK_SIGNS_DATAFRAME.to_csv(STOCK_SIGNS_CSV_LOCATION, index=False)

### FIX 5.19.2
STOCK_SIGNS_DATAFRAME.loc[STOCK_SIGNS_DATAFRAME['SIGN'] == '5.19.2', 'SIGN'] = '5.19.1'
### /FIX 5.19.2
STOCK_SIGNS_DATAFRAME[::6]

In [ ]:
from sklearn import preprocessing

LE_LOCATION = DATA_DIR / 'STOCK_SIGNS_LE.npy'
le = preprocessing.LabelEncoder()

if os.path.isfile(LE_LOCATION):
    le.classes_ = np.load(LE_LOCATION, allow_pickle=True)
else:
    le.fit_transform(STOCK_SIGNS_DATAFRAME['SIGN'])
    np.save(LE_LOCATION, le.classes_)

STOCK_SIGNS_DATAFRAME['ENCODED_LABELS'] = le.transform(STOCK_SIGNS_DATAFRAME['SIGN'])
STOCK_SIGNS_DATAFRAME['filepath'] = STOCK_SIGNS_DATAFRAME['filepath'].apply(lambda x: DATA_DIR / x)
STOCK_SIGNS_DATAFRAME[::6]

In [ ]:
SIGN_LIST = list(STOCK_SIGNS_DATAFRAME['SIGN'])

nrows, ncols = 20, 6
fig = plt.figure(figsize = (16,50))

# for idx, sign_class in enumerate(STOCK_SIGNS_DATAFRAME['SIGN']):
for idx, row in enumerate(STOCK_SIGNS_DATAFRAME.iterrows()):
    instance = row[1]
    # print(instance)
    # instance = STOCK_SIGNS_DATAFRAME[STOCK_SIGNS_DATAFRAME['SIGN'] == sign_class].iloc[-1]
    path = str(instance['filepath'])
    sign = instance['SIGN']
    encoded_label = instance['ENCODED_LABELS']
    
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    trans_mask = img[:,:,3] == 0
    img[trans_mask] = [random.randrange(0, 256), random.randrange(0, 256), random.randrange(0, 256), 255]
    img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
    ax = fig.add_subplot(nrows, ncols, idx+1)
    
    ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), aspect=1)
    ax.set_title('ENCODED: ' + str(encoded_label) 
                 + '\nSIGN: ' + str(sign)
                )
    
plt.tight_layout()

Создадим загрузчик с аугментацией

In [ ]:
class SignDataset(torch.utils.data.Dataset):
    def __init__(self, df, set_label=None, img_size=64, hyp=None, transform=None, le=None):
        
        if isinstance(img_size, int):
            img_size = (img_size, img_size)
        self.img_size = img_size
        
        self.transform = transform
        
        if set_label == None:
            self.df = df
        else:
            self.df = df[df['SET']==set_label]
        
        self.hyp = hyp

    def __len__(self):
        return len(self.df.index)
    
    def __getitem__(self, index): 
        label = int(self.df.iloc[index]['ENCODED_LABELS'])
        path = str(self.df.iloc[index]['filepath'])
        
        img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
        # check does it contains transparent channel 
        if img.shape[2] == 4:
        # randomize transparent
            trans_mask = img[:,:,3] == 0
            img[trans_mask] = [random.randrange(0, 256), 
                               random.randrange(0, 256), 
                               random.randrange(0, 256), 255]

            img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
        # /randomize transparent
        
        img = cv2.resize(img, self.img_size, interpolation=cv2.INTER_LANCZOS4)
        
        # augment
        if self.transform:
            img = self.transform(image=img)['image']
            
        if self.hyp and self.transform:
            img, _ =  random_perspective(img, 
                                      (),
                                      degrees=self.hyp['degrees'],
                                      translate=self.hyp['translate'],
                                      scale=self.hyp['scale'],
                                      shear=self.hyp['shear'],
                                      perspective=self.hyp['perspective'],
                                      border=self.hyp['border'])
        
        # /augment
        img = cv2.resize(img, self.img_size, interpolation=cv2.INTER_LANCZOS4)
        
        img_tnsr = torch.Tensor.permute(torch.Tensor(img), [2, 0, 1]).div(255)
        return img_tnsr, label

In [ ]:
img_size = 64

import albumentations as A
transform = A.Compose([
    A.RandomCrop(p=0.2, width=int(img_size*0.9), height=int(img_size*0.9)),
    A.Blur(blur_limit=2, p=0.21),
    # A.MedianBlur(p=0.2),
    A.ToGray(p=0.05),
    # A.CLAHE(p=0.01),
    # A.RandomBrightnessContrast(p=0.0),
    # A.RandomGamma(p=0.01),
    A.ImageCompression(quality_lower=40, p=0.2),
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.2),
    
    ]
)

hyp = {
    'degrees': 10,
    'translate': 0.0,
    'scale': 0.1,
    'shear': 5,
    'perspective': 0.00001,
    'border': (0, 0)
}
train_dataset = SignDataset(STOCK_SIGNS_DATAFRAME, 
                            set_label=None, 
                            img_size=img_size, 
                            transform=transform, 
                            hyp=hyp)
# train_dataset[0]
nrows, ncols = 70, 6
fig = plt.figure(figsize = (16,200))

for idx, (img, encoded_label) in enumerate(train_dataset):
    
    img = torch.Tensor.permute(img, [1, 2, 0]).numpy() 
    ax = fig.add_subplot(nrows, ncols, idx+1)
    ax.patch.set_linewidth('20')
    
    ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), aspect=1)
    ax.set_title(str(le.inverse_transform([encoded_label])[0]), fontsize=15)
    
plt.tight_layout()

Тренимся только на этом датасете. Валидационный возьмем gt.

In [ ]:
gt = pd.read_csv(DATA_DIR / 'RTDS_DATASET.csv')
NUMBER_TO_SIGN = pd.read_csv(DATA_DIR / 'number_to_sign.csv', index_col=0).T.to_dict('records')[0]

gt['SIGN'] = gt['SIGN'].apply(lambda x: NUMBER_TO_SIGN[x].replace('_', '.').replace('n', ''))
gt['filepath'] = gt['filepath'].apply(lambda x: DATA_DIR / x)
gt.loc[gt['SIGN'] == '2.3', 'SIGN'] = '2.3.1'
gt.loc[gt['SIGN'] == '3.18', 'SIGN'] = '3.18.1'

gt['ENCODED_LABELS'] = le.transform(gt['SIGN'])
GT_SIGNS_SET  = set(gt['SIGN'])
STOCK_SIGNS_SET = set(STOCK_SIGNS_DATAFRAME['SIGN'])

In [ ]:
print(GT_SIGNS_SET.difference(STOCK_SIGNS_SET))

STOCK_SIGNS_SET.difference(GT_SIGNS_SET) - пустой, т.к. пак дефолтных изображений содержит всех представителей знаков RTDS.

In [ ]:
valid_dataset = SignDataset(gt, 
                            set_label='test',
                            img_size=img_size
                           )

num_workers=0
if IN_COLAB or USE_COLAB_GPU:
    num_workers=4

batch_size = 260
if IN_COLAB or USE_COLAB_GPU:
    batch_size = 2500
    
train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=True,
        shuffle=False)

valid_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=True,
        shuffle=False)

In [ ]:
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm

def train_epoch(model, loader, loss_op, optim, device):

    model.train()
    model.to(device)
    
    losses = 0
    rights = 0
    pbar = tqdm(enumerate(loader),
                total=len(loader), 
                position=0,
                leave=False)
    
    for idx, (data, target) in pbar:
                    
        data = data.to(device)
        target = target.to(device)

        optim.zero_grad()
        pred = model(data)

        iter_right_count = get_rights_count(pred, target).cpu().numpy()
        rights += iter_right_count
        
        loss = loss_op(pred, target)
        losses += loss.item()
        
        # Gradient descent
        loss.backward()
        optim.step()
        
        pbar.set_description("TRAIN: INSTANT LOSS %f INSTANT ACCUR: %.4f" % 
                             (round(loss.item(), 3), 
                              iter_right_count / len(target))
                            )
        
    return losses

def get_rights_count(y_pred, y_true):
    y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
    _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)    
    correct_pred = (y_pred_tags == y_true).float()
    acc = correct_pred.sum()
    return acc

def valid_epoch(model, loader, device):
    
    model.eval()
    model.to(device)
    
    rights = 0
    pbar = tqdm(enumerate(loader),
                total=len(loader),
                position=0,
                leave=False)
        
    for idx, (data, target) in pbar:
        data = data.to(device)
            
        target = target.to(device)
        pred = model(data)

        iter_right_count = get_rights_count(pred, target).cpu().numpy()
        rights += iter_right_count
        
        pbar.set_description("VALIDATION: INSTANT ACCUR: %.4f" % (iter_right_count / len(target)))
        
    return rights / len(loader.dataset)

In [ ]:
SHOULD_I_TRAIN = True

config = {
    'lr': 0.1,
    'epochs': 5,
}

DEFAULT_MODEL_LOCATION = DATA_DIR / 'CLASSIFIER_ON_STOCK'

from torchvision import models
model = models.resnet18(pretrained=True)
MODEL_CLASSES = len(STOCK_SIGNS_SET)
model.fc = nn.Sequential(
    nn.Linear(512, MODEL_CLASSES),
    nn.Softmax(dim=1)
)

model.eval()    

loss_op = nn.CrossEntropyLoss().to(device)
optim = torch.optim.Adadelta(model.parameters(), lr=config['lr'])

model.to(device)

if SHOULD_I_TRAIN:
    pbar = tqdm(range(config['epochs']),
                total=config['epochs'],
                position=0,
                leave=True,
                desc='WAITING FOR FIRST EPOCH END...')
    
    for epoch in pbar:

        train_loss = train_epoch(model, train_loader, loss_op, optim, device)
        mean_train_acc = valid_epoch(model, train_loader, device)        
        mean_valid_acc = valid_epoch(model, valid_loader, device)
                
        torch.save(model.state_dict(), DEFAULT_MODEL_LOCATION)
        model_save_name = 'CLASSIFIER_ON_STOCK_{}_TRAIN_ACC{:.4f}_VALID_ACC{:.4f}'.format(datetime.now().strftime("%m.%d_%H.%M"),
                                                                      mean_train_acc,
                                                                      mean_valid_acc)    
        
        torch.save(model.state_dict(), DATA_DIR / model_save_name)
        if IN_COLAB:
            shutil.copy2(DATA_DIR / model_save_name, '/content/drive/MyDrive/')

            
        pbar.set_description("PER EPOCH: TRAIN LOSS: %4f; TRAIN ACCUR %.4f; VALID ACCUR: %.4f" % (train_loss, 
                                                                                                   mean_train_acc,
                                                                                                   mean_valid_acc)
                            )

    print("END TRAIN ACCUR: %.4f; VALID ACCUR %.4f" % (mean_train_acc, mean_valid_acc))
else:
    print('SHOULD I TRAIN == FALSE, SKIP TRAINING')

In [ ]:
img

In [ ]:
transform(img, None)

In [ ]:
[PREFIX + sign, '.'.join(sign.split('.')[:-1])]

In [ ]:
pd.DataFrame(data=[[PREFIX + sign, '.'.join(sign.split('.')[:-1])]], columns=STOCK_SIGNS_DATAFRAME.columns)

In [ ]:
'.'.join(sign.split('.')[:-1])

In [ ]:
PREFIX+sign

In [ ]:
STOCK_SIGNS_LIST